# WEB SCRAPING

### Código utilizado durante el proceso de extracción de imágenes mediante web scraping

Web scrapeada: https://www.moto1pro.com/

In [1]:
from bs4 import BeautifulSoup
import requests
from lxml import html
from urllib.parse import urljoin
from selenium import webdriver
from urllib.request import urlretrieve
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import warnings
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait   
from selenium.webdriver.support import expected_conditions as EC
import csv
warnings.filterwarnings('ignore')
options=Options()

options.add_extension('../driver/adblock.crx')
options.add_argument('--start-maximized')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
#options.add_argument('--incognito')
#options.add_argument('user-data-dir=selenium')

In [2]:
PATH = ChromeDriverManager().install()

In [3]:
driver = webdriver.Chrome(PATH, options=options)

In [4]:
url = 'https://www.moto1pro.com/'

In [5]:
driver.get(url)

In [6]:
# Acepto las cookies
cookies = driver.find_element(By.XPATH, '/html/body/div[11]/div[2]/div[1]/div[2]/div[2]/button[1]/p')
cookies.click()
time.sleep(0.5)

In [7]:
#Accedo a las fichas tecnicas
fichas = driver.find_elements(By.XPATH, '//*[@id="navWrapper"]/li[6]/a')
fichas[0].click()
time.sleep(0.5)

TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=111.0.5563.64)
Stacktrace:
Backtrace:
	(No symbol) [0x00EADCE3]
	(No symbol) [0x00E439D1]
	(No symbol) [0x00D54DA8]
	(No symbol) [0x00D480AF]
	(No symbol) [0x00D47EFC]
	(No symbol) [0x00D46BF6]
	(No symbol) [0x00D4747A]
	(No symbol) [0x00D4FF62]
	(No symbol) [0x00D5A348]
	(No symbol) [0x00D5CD46]
	(No symbol) [0x00D477D3]
	(No symbol) [0x00D5A211]
	(No symbol) [0x00DAD779]
	(No symbol) [0x00D9ACC6]
	(No symbol) [0x00D76F68]
	(No symbol) [0x00D780CD]
	GetHandleVerifier [0x01123832+2506274]
	GetHandleVerifier [0x01159794+2727300]
	GetHandleVerifier [0x0115E36C+2746716]
	GetHandleVerifier [0x00F56690+617600]
	(No symbol) [0x00E4C712]
	(No symbol) [0x00E51FF8]
	(No symbol) [0x00E520DB]
	(No symbol) [0x00E5C63B]
	BaseThreadInitThunk [0x776800F9+25]
	RtlGetAppContainerNamedObjectPath [0x77EB7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77EB7B8E+238]


In [ ]:
elems = driver.find_elements(By.XPATH, "//a[@href]")

In [ ]:
links = []
for elem in elems:
    links.append(elem.get_attribute("href"))

In [ ]:
links = [elem.get_attribute("href") for elem in elems]

In [ ]:
links_marcas = []             
for e in links:
    if 'marca-de-motos/' in e:
        links_marcas.append(e)
#links_marcas

In [ ]:
url_base = "https://www.moto1pro.com/marca-de-motos/"                                    
marcas = ['Aprilia', 'Bmw', 'Ducati', 'Honda', 'Yamaha', 'Kawasaki', 'Suzuki', 'KTM']

full_links = []                                                           
visited_links = set()

for marca in marcas:
    url = urljoin(url_base, marca)
    response = requests.get(url)
    tree = html.fromstring(response.content)
    div_content = tree.xpath('//div[@class="contenido-izquierda"]')[0]
    links = div_content.xpath('.//a/@href')
    for link in links:
        full_link = urljoin(url, link)
        if full_link not in visited_links:
            full_links.append(full_link)
            visited_links.add(full_link)
#full_links

In [ ]:
url_base = "https://www.moto1pro.com/marca-de-motos"
marcas = ['Aprilia', 'Bmw', 'Ducati', 'Honda', 'Yamaha', 'Kawasaki', 'Suzuki', 'KTM']

image_links = []
for marca in marcas:
    full_link = urljoin(url_base, f"marca-de-motos/{marca}")
    r = requests.get(full_link)
    soup = BeautifulSoup(r.text, 'lxml')
    try:
        elements_with_image = soup.find_all('img', {'class': 'img-responsive'})
        for element in elements_with_image:
            image_links.append(element['src'])
    except:
        print(f"No se pudieron obtener los datos para la marca: {marca}")

df = pd.DataFrame({'image_links': image_links})

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df_images = df

In [ ]:
df_images

In [ ]:
#df_images.to_csv('image_URL.csv')

In [ ]:
#df_images.to_csv('nuevo.csv', index=True)